<programa> ::= <declaracoes> <atribuicoes>

<declaracoes> ::= <declaracao> |
                   <declaracao> <declaracoes>

<declaracao> ::= variavel texto |
                  variavel inteiro

<atribuicoes> ::= <atribuicao> |
                   <atribuicao> <atribuicoes>

<atribuicao> ::= variavel = <expressao>

<expressao> ::= variavel |
                 numero |
                 <expressao> mais <expressao>
                 <expressao> menos <expressao>
                 <expressao> dividido_por <expressao>
                 <expressao> multiplicado_por <expressao>

<se-senao> ::= se <expressao> <comparador> <expressao> faz <atribuicao> ; |
                se <expressao> <comparador> <expressao> faz <se-senao>; |
                se <expressao> <comparador> <expressao> faz <atribuicao> caso_contrario_faz <atribuicao>; |
                se <expressao> <comparador> <expressao> faz <atribuicao> caso_contrario_faz <se-senao>; |
                se <expressao> <comparador> <expressao> faz <se-senao> caso_contrario_faz <atribuicao>; |
                se <expressao> <comparador> <expressao> faz <se-senao> caso_contrario_faz <se-senao>; |
